<a href="https://colab.research.google.com/github/raaz0000002/pytorch_learning/blob/main/Tiles_Creating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-image


In [ ]:
!pip install gdown


In [ ]:
pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 20.9 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from tqdm import tqdm

def create_tiles_tif(image_path, output_dir, tile_size=(512,512), prefix="tile"):
    """
    Splits a TIFF image into smaller tiles and saves them as .tif files.

    Args:
        image_path (str): Path to input .tif image.
        output_dir (str): Directory to save tiles.
        tile_size (tuple): Size of each tile (height, width).
        prefix (str): Prefix for tile filenames.
    """
    os.makedirs(output_dir, exist_ok=True)

    with rasterio.open(image_path) as src:
        width = src.width
        height = src.height
        num_bands = src.count
        profile = src.profile

        tile_height, tile_width = tile_size
        tile_id = 0

        for top in tqdm(range(0, height, tile_height), desc="Creating Tiles"):
            for left in range(0, width, tile_width):
                # Ensure tile doesn't exceed bounds
                w = min(tile_width, width - left)
                h = min(tile_height, height - top)

                window = Window(left, top, w, h)
                transform = src.window_transform(window)
                tile_data = src.read(window=window)

                # Update profile for tile
                tile_profile = profile.copy()
                tile_profile.update({
                    'height': h,
                    'width': w,
                    'transform': transform
                })

                tile_filename = os.path.join(output_dir, f"{prefix}_{tile_id:04d}.tif")
                with rasterio.open(tile_filename, 'w', **tile_profile) as dst:
                    dst.write(tile_data)

                tile_id += 1




In [ ]:
import gdown
file_id = "1UIJOqsp1cWhnIfFbL3XF1CerfqAc_7U1"
url = f"https://drive.google.com/uc?id={file_id}"
output = "/content/satellite_image.tif"

gdown.download(url, output, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1UIJOqsp1cWhnIfFbL3XF1CerfqAc_7U1
From (redirected): https://drive.google.com/uc?id=1UIJOqsp1cWhnIfFbL3XF1CerfqAc_7U1&confirm=t&uuid=e3bccb28-0d3f-4253-9184-83a1f12cb1cb
To: /content/satellite_image.tif
100%|██████████| 6.90G/6.90G [01:50<00:00, 62.4MB/s]


'/content/satellite_image.tif'

In [ ]:
import os
import gdown

# Step 2: Tile output folder
output_tiles_folder = "/content/drive/MyDrive/Ideathon/Untitled Folder"

# Step 3: Call the tiling function
create_tiles_tif(output, output_tiles_folder)



Creating Tiles:   8%|▊         | 12/158 [00:44<08:59,  3.70s/it]


KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import rasterio
from skimage.io import imsave
from tqdm import tqdm

def normalize_to_uint8(image, percentile_low=2, percentile_high=98, gamma=1.0):
    """
    Normalize image to 0–255 with percentile clipping and optional gamma correction.

    Args:
        image: Input image as numpy array.
        percentile_low: Lower percentile for clipping (e.g., 2).
        percentile_high: Upper percentile for clipping (e.g., 98).
        gamma: Gamma correction factor (e.g., 0.5 to brighten, >1 to darken).
    """
    image = image.astype(np.float32)

    # Use percentiles to clip outliers
    low_val = np.percentile(image, percentile_low)
    high_val = np.percentile(image, percentile_high)

    # Clip the image to the percentile range
    image = np.clip(image, low_val, high_val)

    # Normalize to 0-1
    image = (image - low_val) / (high_val - low_val)

    # Apply gamma correction (optional)
    if gamma != 1.0:
        image = np.power(image, gamma)

    # Scale to 0-255 and convert to uint8
    image = image * 255
    return image.astype(np.uint8)

def create_tiles_from_tif(tif_path, output_folder, tile_size=(512, 512), stride=(512, 512), gamma=0.5):
    """
    Splits a TIFF image into tiles and saves as PNGs with brightness adjustment.

    Args:
        tif_path (str): Path to local .tif image.
        output_folder (str): Where to save the PNG tiles.
        tile_size (tuple): (Height, Width) of each tile.
        stride (tuple): Stride between tiles (for overlap).
        gamma (float): Gamma correction value (e.g., 0.5 to brighten).
    """
    os.makedirs(output_folder, exist_ok=True)

    try:
        with rasterio.open(tif_path) as src:
            print(f"TIFF dimensions: {src.height}x{src.width}, Bands: {src.count}")

            if src.count >= 3:
                image = src.read([1, 2, 3])  # Read RGB bands
            else:
                image = src.read(1)  # Read single band
                image = np.stack([image] * 3, axis=0)  # Stack to create RGB

            image = np.transpose(image, (1, 2, 0))  # HWC

            # Normalize with percentile clipping and gamma correction
            image = normalize_to_uint8(image, percentile_low=2, percentile_high=98, gamma=gamma)

            if image.max() == 0:
                raise ValueError("Image data is all zeros after normalization")

            H, W, _ = image.shape
            tile_H, tile_W = tile_size

            print(f"Image shape after processing: {image.shape}")

            tile_id = 0
            for i in tqdm(range(0, H - tile_H + 1, stride[0]), desc="Tiling rows"):
                for j in range(0, W - tile_W + 1, stride[1]):
                    tile = image[i:i + tile_H, j:j + tile_W]
                    if tile.max() == 0:
                        print(f"Skipping blank tile at {i},{j}")
                        continue

                    save_path = os.path.join(output_folder, f"tile_{tile_id:04d}.jpg")
                    imsave(save_path, tile, check_contrast=False)
                    tile_id += 1

            print(f"Generated {tile_id} tiles")

    except Exception as e:
        print(f"Error processing TIFF file: {str(e)}")

# ========== USER INPUT ==========
input_tif_path = "/content/drive/MyDrive/Thesis/tif2.tif"  # Your TIFF path
output_folder = "/content/drive/MyDrive/Ideathon/jpg_tiles"     # Your output folder

# ========== RUN ==========
create_tiles_from_tif(input_tif_path, output_folder, gamma=1.06)  # Gamma < 1 brightens

TIFF dimensions: 5088x6242, Bands: 5
Image shape after processing: (5088, 6242, 3)


Tiling rows:   0%|          | 0/9 [00:00<?, ?it/s]

Skipping blank tile at 0,0
Skipping blank tile at 0,512
Skipping blank tile at 0,3584
Skipping blank tile at 0,4096
Skipping blank tile at 0,4608
Skipping blank tile at 0,5120
Skipping blank tile at 0,5632
Skipping blank tile at 512,0


Tiling rows:  33%|███▎      | 3/9 [00:00<00:00,  7.22it/s]

Skipping blank tile at 512,5120
Skipping blank tile at 512,5632
Skipping blank tile at 1024,0
Skipping blank tile at 1024,5632
Skipping blank tile at 1536,0


...

Skipping blank tile at 2048,0
Skipping blank tile at 2048,512
Skipping blank tile at 2560,0
Skipping blank tile at 2560,512


Tiling rows:  78%|███████▊  | 7/9 [00:01<00:00,  5.96it/s]

Skipping blank tile at 3072,0
Skipping blank tile at 3072,512
Skipping blank tile at 3584,0
Skipping blank tile at 3584,512


Tiling rows: 100%|██████████| 9/9 [00:01<00:00,  6.23it/s]

Skipping blank tile at 4096,0
Skipping blank tile at 4096,512
Generated 85 tiles


In [ ]:
# Install necessary libraries if not already installed
!pip install scikit-image
!pip install gdown
!pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 64.0 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from tqdm import tqdm
from skimage.io import imsave


def normalize_to_uint8(image, percentile_low=2, percentile_high=98, gamma=1.0):
    image = image.astype(np.float32)
    low_val = np.percentile(image, percentile_low)
    high_val = np.percentile(image, percentile_high)
    image = np.clip(image, low_val, high_val)
    image = (image - low_val) / (high_val - low_val + 1e-5)
    if gamma != 1.0:
        image = np.power(image, gamma)
    image = image * 255
    return image.astype(np.uint8)


def create_jpg_tiles_from_tif(tif_path, output_folder, tile_size=(512, 512), stride=(512, 512), gamma=1.0):
    os.makedirs(output_folder, exist_ok=True)

    with rasterio.open(tif_path) as src:
        print(f"📂 Input TIFF: {tif_path}")
        print(f"🖼️ Dimensions: {src.height}x{src.width}, Bands: {src.count}")

        # Read RGB bands: B2 (band 1), B3 (band 2), B4 (band 3)
        data = src.read([1, 2, 3])  # shape: (3, H, W)
        data = np.transpose(data, (1, 2, 0))  # shape: (H, W, 3)

        # Replace no-data or extreme values with NaN
        data[data <= -1e+10] = np.nan
        data = np.nan_to_num(data)

        # Normalize to 0-255 uint8
        data = normalize_to_uint8(data, gamma=gamma)

        H, W, _ = data.shape
        tile_H, tile_W = tile_size
        tile_id = 0

        for i in tqdm(range(0, H - tile_H + 1, stride[0]), desc="Tiling rows"):
            for j in range(0, W - tile_W + 1, stride[1]):
                tile = data[i:i + tile_H, j:j + tile_W]
                if tile.max() <= 5:  # Skip empty/near-empty tiles
                    continue
                save_path = os.path.join(output_folder, f"tile_{tile_id:04d}.jpg")
                imsave(save_path, tile, check_contrast=False)
                tile_id += 1

        print(f"✅ Done! {tile_id} tiles saved to: {output_folder}")


# 👉 Run this section only if you're in an environment like Google Colab and Drive is mounted
tif_path = "/content/drive/MyDrive/Thesis/tif2.tif"
output_folder = "/content/drive/MyDrive/Ideathon/Tiles"

create_jpg_tiles_from_tif(tif_path, output_folder, gamma=1.06)


📂 Input TIFF: /content/drive/MyDrive/Thesis/tif2.tif
🖼️ Dimensions: 5088x6242, Bands: 5


Tiling rows: 100%|██████████| 9/9 [00:01<00:00,  6.19it/s]


✅ Done! 85 tiles saved to: /content/drive/MyDrive/Ideathon/Tiles


In [ ]:
import rasterio

def inspect_tiff_bands(tif_path):
    with rasterio.open(tif_path) as src:
        print(f"📂 File: {tif_path}")
        print(f"🖼️ Dimensions: {src.height}x{src.width}")
        print(f"📊 Number of Bands: {src.count}")
        print(f"🧭 CRS: {src.crs}")
        print(f"📐 Transform: {src.transform}")
        print("\n🔍 Band Information:")

        for i in range(1, src.count + 1):  # 1-based indexing
            desc = src.descriptions[i-1] if src.descriptions[i-1] else "N/A"
            stats = src.read(i)
            print(f" Band {i}: {desc}")
            print(f"   - Min: {stats.min()}, Max: {stats.max()}, Mean: {stats.mean():.2f}")

# === USAGE ===
tif_path = "/content/drive/MyDrive/Thesis/tif2.tif"
inspect_tiff_bands(tif_path)


📂 File: /content/drive/MyDrive/Thesis/tif2.tif
🖼️ Dimensions: 5088x6242
📊 Number of Bands: 5
🧭 CRS: EPSG:4326
📐 Transform: | 0.00, 0.00, 86.45|
| 0.00,-0.00, 28.10|
| 0.00, 0.00, 1.00|

🔍 Band Information:
 Band 1: B2
   - Min: -3.4028230607370965e+38, Max: 1.392699956893921, Mean: -inf
 Band 2: B3
   - Min: -3.4028230607370965e+38, Max: 1.3668999671936035, Mean: -inf
 Band 3: B4


/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


   - Min: -3.4028230607370965e+38, Max: 1.3971500396728516, Mean: -inf
 Band 4: B8
   - Min: -3.4028230607370965e+38, Max: 1.305400013923645, Mean: -inf
 Band 5: QA60
   - Min: -3.4028230607370965e+38, Max: 0.0, Mean: -inf
